<a href="https://colab.research.google.com/github/NicholsonCameron/Lab-1.1/blob/main/comparing_league_wide_ERA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import polars as pl

In [65]:
pitching = pl.read_csv('./Data/lahman_1871-2023_csv/Pitching.csv')
pitching.head()

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2


In [111]:
ERA = (pitching 
       .select([pl.col('IPouts'), pl.col('R'), pl.col('lgID'), pl.col('yearID')])
       .filter(pl.col('yearID') >= 1946)
       .group_by(pl.col('yearID'), pl.col('lgID'))
       .agg([pl.col('IPouts').sum().alias('total_outs'),
             pl.col('R').sum().alias('total_runs'),
            ])
       .with_columns(era = (pl.col('total_runs')/pl.col('total_outs')*27),
                    DH_rule =  pl.when(pl.col('yearID') < 1973)
                                 .then(pl.lit('No'))
                                 .when(pl.col('lgID') == 'AL')
                                 .then(pl.lit('Yes'))
                                 .when(pl.col('yearID') > 2022)
                                 .then(pl.lit('Yes'))
                                 .when(pl.col('lgID') == 'NL')
                                 .then(pl.lit('No'))
                    )
        )
ERA.head()

yearID,lgID,total_outs,total_runs,era,DH_rule
i64,str,i64,i64,f64,str
2020,"""NL""",23068,4185,4.898344,"""No"""
2011,"""AL""",60850,10036,4.453114,"""Yes"""
2014,"""AL""",65396,10052,4.150162,"""Yes"""
2012,"""AL""",60855,9980,4.427902,"""Yes"""
1992,"""NL""",52502,7539,3.877052,"""No"""


In [108]:
ERA.write_csv('./data/ERA.csv')

In [67]:
batting = pl.read_csv('./Data/lahman_1871-2023_csv/Batting.csv')
batting.head()

playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""aardsda01""",2004,1,"""SFN""","""NL""",11,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2006,1,"""CHN""","""NL""",45,null,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,null
"""aardsda01""",2007,1,"""CHA""","""AL""",25,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2008,1,"""BOS""","""AL""",47,null,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,null
"""aardsda01""",2009,1,"""SEA""","""AL""",73,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null


In [109]:
SLG = (batting
       .select([pl.col('BB'), pl.col('2B'), pl.col('3B'), pl.col('HR'), 
                pl.col('AB'), pl.col('yearID'), pl.col('lgID')])
       .filter(pl.col('yearID') >= 1946)
       .group_by(pl.col('yearID'), pl.col('lgID'))
       .agg([pl.col('BB').sum().alias('total_BB'),
             pl.col('2B').sum().alias('total_2B'),
             pl.col('3B').sum().alias('total_3B'),
             pl.col('HR').sum().alias('total_HR'),
             pl.col('AB').sum().alias('total_AB'),
            ])
       .with_columns(slg = (pl.col('total_BB') + (2 * pl.col('total_2B')) + 
                            (3 * pl.col('total_3B')) + (4 * pl.col('total_HR')))/pl.col('total_AB'),
                    DH_rule =  pl.when(pl.col('yearID') < 1973)
                                 .then(pl.lit('No'))
                                 .when(pl.col('lgID') == 'AL')
                                 .then(pl.lit('Yes'))
                                 .when(pl.col('yearID') > 2022)
                                 .then(pl.lit('Yes'))
                                 .when(pl.col('lgID') == 'NL')
                                 .then(pl.lit('No'))
                    )
      )
SLG.head()

yearID,lgID,total_BB,total_2B,total_3B,total_HR,total_AB,slg,DH_rule
i64,str,i64,i64,i64,i64,i64,f64,str
1946,"""NL""",4400,1752,382,562,42094,0.268399,"""No"""
2020,"""NL""",3078,1402,126,1161,29399,0.370897,"""No"""
2006,"""NL""",8600,4834,561,2840,88844,0.352427,"""No"""
1979,"""AL""",7413,3529,548,2006,76704,0.314703,"""Yes"""
2007,"""AL""",7503,4299,433,2252,78294,0.337293,"""Yes"""


In [110]:
SLG.write_csv('./data/SLG.csv')